In [1]:
import pandas as pd
import copy
import torch
from typing import List, Dict, Tuple
from datasets import load_dataset, Dataset
from dotenv import load_dotenv
# Use a pipeline as a high-level helper
from transformers import pipeline
from tqdm.notebook import tqdm

load_dotenv()
pipe = pipeline("token-classification", model="obi/deid_bert_i2b2")

Device set to use mps:0


In [2]:
# df = pd.read_csv('healthcare_dataset.csv')

In [3]:
dataset = load_dataset("mks-logic/SPY", trust_remote_code=True, faker_random_seed=42)

In [4]:
# dataset

In [5]:
for x in dataset['medical_consultations']:
    # print(x)
    break
print(dataset.column_names)
lst_of_ent_tags = dataset['medical_consultations']['ent_tags']
lst_of_tokens = dataset['medical_consultations']['tokens']
lst_of_trailing_whitespaces = dataset['medical_consultations']['trailing_whitespaces']
lst_of_labels = dataset['medical_consultations']['labels']
print(type(lst_of_ent_tags[0][2]))
print(type(lst_of_tokens[0][2]))
print(type(lst_of_trailing_whitespaces[0][2]))
print(type(lst_of_labels[0][2]))

{'legal_questions': ['tokens', 'trailing_whitespaces', 'labels', 'ent_tags'], 'medical_consultations': ['tokens', 'trailing_whitespaces', 'labels', 'ent_tags']}
<class 'str'>
<class 'str'>
<class 'bool'>
<class 'int'>


In [6]:
print(lst_of_tokens[0])
print(lst_of_tokens[0].index("Christopher"))
print(lst_of_trailing_whitespaces[0])
print(lst_of_labels[0])

['Text', ': ', 'Hi', ', ', 'I ', 'am ', 'Christopher', 'Murillo', ', ', '\n', 'I ', 'am ', 'experiencing ', 'a ', 'sharp ', 'pain ', 'in ', 'the ', 'lower ', 'right ', 'abdomen', ', ', 'occasionally ', 'radiating ', 'to ', 'my ', 'back', '. ', 'This ', 'symptom ', 'has ', 'been ', 'persistent ', 'for ', 'the ', 'past ', '4 ', 'days ', 'now', '. ', 'The ', 'pain ', 'is ', 'worsening ', 'over ', 'time', ', ', 'especially ', 'after ', 'eating ', 'or ', 'engaging ', 'in ', 'physical ', 'activities', '. ', 'I ', 'have ', 'noticed ', 'minor ', 'nausea', ', ', 'but ', 'no ', 'vomiting ', 'or ', 'bleeding', '. ', 'The ', 'pain ', 'is ', 'non', '-', 'specific', ', ', 'meaning ', 'it ', 'does', "n't ", 'seem ', 'to ', 'be ', 'triggered ', 'by ', 'a ', 'specific ', 'food ', 'intake ', 'or ', 'any ', 'other ', 'activity', '. ', '\n\n', 'You ', 'can ', 'reach ', 'me ', 'at ', 'alvarezkenneth@gmail.com', 'or ', '493-290-9635', 'for ', 'any ', 'clarification ', 'or ', 'follow', '-', 'up ', 'questions

In [7]:
from typing import List

def reconstruct_text_from_tokens(tokens: List[str], has_trailing_space: List[bool], labels: List[int]) -> str:
    """
    Reconstructs a text string from a list of tokens and a list of trailing whitespace indicators.

    Args:
        tokens (List[str]): List of token strings.
        has_trailing_space (List[bool]): List of booleans where True means a space should follow the token.

    Returns:
        str: The reconstructed string with appropriate single-space separations.
    
    Raises:
        ValueError: If the lengths of tokens and has_trailing_space do not match.
    """
    if len(tokens) != len(has_trailing_space):
        raise ValueError("Length of tokens and has_trailing_space must be equal.")
    
    pieces = []
    for token, has_space, label in zip(tokens, has_trailing_space, labels):
        pieces.append(token)
        if (label != 14):
            pieces.append(" ")

    return "".join(pieces)


In [8]:
t = reconstruct_text_from_tokens(lst_of_tokens[0],lst_of_trailing_whitespaces[0],lst_of_labels[0])
print(t)

Text: Hi, I am Christopher Murillo , 
I am experiencing a sharp pain in the lower right abdomen, occasionally radiating to my back. This symptom has been persistent for the past 4 days now. The pain is worsening over time, especially after eating or engaging in physical activities. I have noticed minor nausea, but no vomiting or bleeding. The pain is non-specific, meaning it doesn't seem to be triggered by a specific food intake or any other activity. 

You can reach me at alvarezkenneth@gmail.com or 493-290-9635 for any clarification or follow-up questions.

I have a history of appendicitis which was surgically removed when I was 17. I have also had several episodes of stomach ulcers in the past, for which I have taken antacids and antibiotics as prescribed by my previous doctor, Dr. Emily Patel, whose contact information is available at https://facebook.com/healthcareprofessionals. 

Currently, I am not taking any medications, but I have an upcoming doctor's appointment with Dr. Soph

In [9]:
lst_of_docs = [" ".join(lst)
    for lst in lst_of_tokens
]
lst_of_docs = [" ".join(doc.split())
              for doc in lst_of_docs]
print(lst_of_docs[1])

Hello ! I 'm Jacqueline Adams , reaching out for an online consultation due to a recurring issue with tooth sensitivity . It 's been going on for the past 6 months , and it 's been getting gradually worse . Sometimes it 's just a mild tingling sensation , but other times it 's a sharp pain . I 've noticed it 's triggered by eating or drinking something hot or cold . I 've had a few dental fillings in the past , but no major dental work . You can reach me back at christopher17@gmail.com if there 's any additional information required . I 've tried using toothpaste for sensitive teeth , but it does n't seem to be making a difference . My dentist prescribed me a fluoride rinse , but I have n't seen much improvement . In my free time , I usually play online games under the username millerchristina . I 'd appreciate it if you could recommend any products or services that are available at 6771 Johns Shores or can be shipped to my location . Could you please advise on what might be causing th

In [10]:
print(dataset['medical_consultations']['tokens'][0])

['Text', ': ', 'Hi', ', ', 'I ', 'am ', 'Christopher', 'Murillo', ', ', '\n', 'I ', 'am ', 'experiencing ', 'a ', 'sharp ', 'pain ', 'in ', 'the ', 'lower ', 'right ', 'abdomen', ', ', 'occasionally ', 'radiating ', 'to ', 'my ', 'back', '. ', 'This ', 'symptom ', 'has ', 'been ', 'persistent ', 'for ', 'the ', 'past ', '4 ', 'days ', 'now', '. ', 'The ', 'pain ', 'is ', 'worsening ', 'over ', 'time', ', ', 'especially ', 'after ', 'eating ', 'or ', 'engaging ', 'in ', 'physical ', 'activities', '. ', 'I ', 'have ', 'noticed ', 'minor ', 'nausea', ', ', 'but ', 'no ', 'vomiting ', 'or ', 'bleeding', '. ', 'The ', 'pain ', 'is ', 'non', '-', 'specific', ', ', 'meaning ', 'it ', 'does', "n't ", 'seem ', 'to ', 'be ', 'triggered ', 'by ', 'a ', 'specific ', 'food ', 'intake ', 'or ', 'any ', 'other ', 'activity', '. ', '\n\n', 'You ', 'can ', 'reach ', 'me ', 'at ', 'alvarezkenneth@gmail.com', 'or ', '493-290-9635', 'for ', 'any ', 'clarification ', 'or ', 'follow', '-', 'up ', 'questions

In [11]:
df = dataset['medical_consultations'].to_pandas()

In [12]:
df['text'] = df['tokens'].apply(lambda x: "".join(x))
dataset_txt=Dataset.from_pandas(df)

In [13]:
#dataset_txt['text']

In [14]:
# Use a pipeline as a high-level helper
#from transformers import pipeline

#pipe = pipeline("token-classification", model="obi/deid_roberta_i2b2")

In [15]:
#results= pipe{dataset['medical_consultations']}

In [36]:
#dataset['medical_consultations']

In [37]:
#results=[]
#for out in pipe(dataset['medical_consultations']):
    
    #results.append(out)

In [38]:
#dataset_v2 = dataset_txt['text'].map(
          #  pipe,
           # batched=True,
            #batch_size=4,
        # )


In [39]:
# pipe(['my name is president Barack Obama'])

In [40]:
# results=[]
# #with torch.inference_mode():
# for inp in dataset_txt['text']:
#     # print(inp)
#     assert isinstance(inp, str), "not a str"
#     out=pipe([inp], truncation=True)
#     results.append(out)

In [41]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("obi/deid_roberta_i2b2")
model = AutoModelForTokenClassification.from_pretrained("obi/deid_roberta_i2b2")

In [42]:
# inputs = tokenizer(inp, truncation=True, return_tensors='pt')

In [43]:
# outputs = model(**inputs)

In [44]:
# outputs.logits
# tokenizer.decode(outputs)


In [16]:
from transformers import AutoTokenizer

tokenizer = pipe.tokenizer  # get tokenizer from your existing pipeline

results = []
for inp in tqdm(lst_of_docs):
    assert isinstance(inp, str), "not a str"
    
    # Truncate the input at token level
    inputs = tokenizer(
        inp,
        truncation=True,
        max_length=512,
        return_tensors='pt',
        return_attention_mask=True
    )

    # Decode back to string after truncation
    truncated_text = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)

    # Pass the truncated string to the pipeline
    out = pipe(truncated_text)
    results.append(out)


  0%|          | 0/4491 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
print(results[1])

In [ ]:
def get_true_entities(tokens, ner_tags):
    """
    this function does something

    Args:
        tokens (List[str]): labeled tokens
        ner_tags (List[str]) Maybe ground truth labels? but idk

    Returns:
        entities (List[dict[str, str])): ????
    """
    # label_map = dataset['medical_consultations'].features['ent_tags'].feature.names
    entities = []
    current_entity = []
    for token, tag in zip(tokens, ner_tags):
        # tag_label = label_map[tag]
        if tag_label.startswith("B-"):
            if current_entity:
                entities.append(" ".join(current_entity))
            current_entity = [token]
        elif tag_label.startswith("I-") and current_entity:
            current_entity.append(token)
        else:
            if current_entity:
                entities.append(" ".join(current_entity))
                current_entity = []
    if current_entity:
        entities.append(" ".join(current_entity))
    return entities

# results3 = pipe(truncated_text)

In [10]:
get_true_entities(lst_of_ent_tags, results)

NameError: name 'lst_of_ent_tags' is not defined

In [11]:
lst_of_lst_of_dict = copy.deepcopy(lst_of_ent_tags)
tn,tp = 0,0
fn,fp = 0,0
for i, lst in tqdm(enumerate(results)):
    for dictionary in lst:
        start_idx = dictionary['start']
        end_idx = dictionary ['end']
        doc_i = lst_of_docs[i]
        tok = doc_i[start_idx:end_idx+1]
        ent_tags = lst_of_ent_tags[i]
        entity = dictionary['start']
        doc_tok = lst_of_tokens[i]
        try:
            
            tok_idx = doc_tok.index(tok.strip())
        except ValueError as ve:
            fp += 1
            continue
# print(start_idx)
# print(end_idx)
# print(doc_i)
# print(tok)
# print(ent_tags)
# print(entity)
# print(doc_tok)
# raise ValueError(f'{start_idx} oh no') from ve

        try:
            
            tag = ent_tags[tok_idx]
        except ValueError as ve:
            print(doc_tok)
            print(ent_tags)
            assert len(doc_tok) == len(ent_tags), f'{len(doc_tok)} != {len(ent_tags)}'
        is_pii = tag != 'O'
        if is_pii: 
            tp += 1 
        else:
            
            fp += 1
        # print(tag)
        # lst_of_lst_of_dict = 
        lst_of_lst_of_dict[i].remove(tag)
    for tag in lst_of_lst_of_dict[i]: 
        if tag == 'O':
            
            tn += 1
        else:
             
             fn += 1
            

NameError: name 'copy' is not defined

## new code

In [12]:
lst_of_lst_of_dict = copy.deepcopy(lst_of_ent_tags)
tn,tp = 0,0
fn,fp = 0,0
for i, lst in tqdm(enumerate(results)):
    for dictionary in lst:
        start_idx = dictionary['start']
        end_idx = dictionary ['end']
        doc_i = lst_of_docs[i]
        tok = doc_i[start_idx:end_idx+1]
        ent_tags = lst_of_ent_tags[i]
        entity = dictionary['start']
        doc_tok = lst_of_tokens[i]
        try:
            
            tok_idx = doc_tok.index(tok.strip())
        except ValueError as ve:
            fp += 1
            continue
# print(start_idx)
# print(end_idx)
# print(doc_i)
# print(tok)
# print(ent_tags)
# print(entity)
# print(doc_tok)
# raise ValueError(f'{start_idx} oh no') from ve

        try:
            
            tag = ent_tags[tok_idx]
        except ValueError as ve:
            print(doc_tok)
            print(ent_tags)
            assert len(doc_tok) == len(ent_tags), f'{len(doc_tok)} != {len(ent_tags)}'
        is_pii = tag != 'O'
        if is_pii: 
            tp += 1 
        else:
            
            fp += 1
        # print(tag)
        # lst_of_lst_of_dict = 
        lst_of_lst_of_dict[i].remove(tag)
    for tag in lst_of_lst_of_dict[i]: 
        if tag == 'O':
            
            tn += 1
        else:
             
             fn += 1

NameError: name 'copy' is not defined

In [13]:
i = 0
tp, fp = 0, 0
for lst_of_pii, pii in zip(results, lst_of_tokenized_pii):
    for dictionary in lst_of_pii:
        word = dictionary['word']
        if word in pii:
            tp += 1
        else:
            fp += 1 
        lst_of_tokenized_docs[i].remove(word)
    i += 1
print(f"TP: {tp}\nFP: {fp}")

NameError: name 'results' is not defined

In [14]:
from typing import List

def reconstruct_text_from_tokens(tokens: List[str], has_trailing_space: List[bool], labels: List[int]) -> str:
    """
    Reconstructs a text string from a list of tokens and a list of trailing whitespace indicators.

    Args:
        tokens (List[str]): List of token strings.
        has_trailing_space (List[bool]): List of booleans where True means a space should follow the token.

    Returns:
        str: The reconstructed string with appropriate single-space separations.
    
    Raises:
        ValueError: If the lengths of tokens and has_trailing_space do not match.
    """
    if len(tokens) != len(has_trailing_space):
        raise ValueError("Length of tokens and has_trailing_space must be equal.")
    
    pieces = []
    for token, has_space, label in zip(tokens, has_trailing_space, labels):
        pieces.append(token)
        if (label != 14):
            pieces.append(" ")

    return "".join(pieces)


In [15]:
lst_of_docs = []
for tok_lst, space_lst, label_lst in zip(lst_of_tokens, lst_of_trailing_whitespaces, lst_of_labels):
    doc = reconstruct_text_from_tokens(tok_lst, space_lst, label_lst)
    lst_of_docs.append(doc)
lst_of_tokenized_docs = []
for doc in lst_of_docs:
    tokenized_doc = tokenizer.tokenize(doc)
    if isinstance(tokenized_doc, list):
        lst_of_tokenized_docs.append(tokenized_doc)
    else:
        lst_of_tokenized_docs.append([tokenized_doc])    

NameError: name 'lst_of_tokens' is not defined

In [16]:
print(f"TP: {tp}\nTN: {tn}\nFP: {fp}\nFN: {fn}")

TP: 0
TN: 0
FP: 0
FN: 0


In [17]:
lst_of_pii = []
for tok_lst, lab_lst in zip(lst_of_tokens, lst_of_labels):
    pii = []
    for tok, lab in zip(tok_lst, lab_lst):
        if lab != 14:
            pii.append(tok)
    lst_of_pii.append(pii)

NameError: name 'lst_of_tokens' is not defined

In [18]:
lst_of_tokenized_pii = []
for lst in lst_of_pii:
    tok_pii = []
    for pii in lst: 
        tok = tokenizer.tokenize(pii)
        if isinstance(tok, list):
            for t in tok: 
                tok_pii.append(t)
        else:
            tok_pii.append(tok)
    lst_of_tokenized_pii.append(tok_pii)
print(len(lst_of_tokenized_pii))
lst_of_tokenized_pii[0]

0


IndexError: list index out of range

In [19]:
i = 0 
fn,tn = 0,0
for lst in copy.deepcopy(lst_of_tokenized_pii): 
    fn += len(lst)
    for tok in lst:
        lst_of_tokenized_docs[i].remove(tok)
    i += 1 
for lst in lst_of_tokenized_docs:
    tn += len(lst)

print(f"FN: {fn}\nTN: {tn}")

NameError: name 'copy' is not defined

In [20]:
#lst_of_pii[0]

In [ ]:
def compute_entity_char_spans(tokens: List[str], trailing_spaces: List[bool]) -> List[Tuple[int, int, str]]:
    """
    Computes the character-level spans for each token and its entity tag.

    Args:
        tokens (List[str]): List of tokens in the original input.
        trailing_spaces (List[bool]): Booleans indicating whether a space follows each token.

    Returns:
        List[Tuple[int, int, str]]: List of (start_char, end_char, token) tuples for each token.
    """
    spans = []
    current_position = 0
    for token, has_space in zip(tokens, trailing_spaces):
        start = current_position
        end = start + len(token)
        spans.append((start, end, token))
        current_position = end + (1 if has_space else 0)
    return spans


def evaluate_predictions(
    results: List[List[Dict]],
    tokens_list: List[List[str]],
    whitespace_list: List[List[bool]],
    ent_tags_list: List[List[str]]
) -> Tuple[int, int, int, int]:
    """
    Evaluates model predictions against ground truth using character span matching.

    Args:
        results (List[List[Dict]]): Model output with predicted PII spans.
        tokens_list (List[List[str]]): List of token sequences per document.
        whitespace_list (List[List[bool]]): List of trailing space indicators per document.
        ent_tags_list (List[List[str]]): Ground truth PII tags per document.

    Returns:
        Tuple[int, int, int, int]: (TP, FP, TN, FN) counts.
    """
    tp = fp = tn = fn = 0

    for i, result in enumerate(results):
        tokens = tokens_list[i]
        trailing_spaces = whitespace_list[i]
        ent_tags = ent_tags_list[i]

        token_spans = compute_entity_char_spans(tokens, trailing_spaces)

        # Ground truth spans
        gt_spans = set()
        for tag, span in zip(ent_tags, token_spans):
            if tag != "O":
                gt_spans.add(span)

        # Predicted spans
        pred_spans = set()
        for entity in result:
            pred_spans.add((entity["start"], entity["end"]))

        # Count TP, FP, FN
        tp += len(pred_spans & gt_spans)
        fp += len(pred_spans - gt_spans)
        fn += len(gt_spans - pred_spans)

        # Count TN (tokens that are neither predicted nor PII)
        total_tokens = len(tokens)
        pred_token_idxs = {
            i for i, span in enumerate(token_spans) if span in pred_spans
        }
        gt_token_idxs = {
            i for i, span in enumerate(token_spans) if span in gt_spans
        }
        for i in range(total_tokens):
            if i not in pred_token_idxs and i not in gt_token_idxs:
                tn += 1

    return tp, fp, tn, fn

In [ ]:
# tp, fp, tn, fn = evaluate_predictions(
#     results=results,
#     tokens_list=lst_of_tokens,
#     whitespace_list=lst_of_trailing_whitespaces,
#     ent_tags_list=lst_of_ent_tags
# )

# print(f"True Positives: {tp}")
# print(f"False Positives: {fp}")
# print(f"True Negatives: {tn}")
# print(f"False Negatives: {fn}")

In [ ]:
def get_token_indices_from_predictions(
    model_outputs: list[dict], text: str, tokens: list[str], whitespaces: list[str]
) -> set[int]:
    """
    Maps character-level model predictions to token indices.

    Args:
        model_outputs: List of model entity predictions for one input.
        text: Original text string used as input to the model.
        tokens: Tokenized version of the text.
        whitespaces: List of trailing whitespace after each token.

    Returns:
        A set of token indices that were predicted as entities.
    """
    char_spans = compute_entity_char_spans(tokens, whitespaces)
    predicted_indices = set()

    for pred in model_outputs:
        pred_start = pred["start"]
        pred_end = pred["end"]

        for idx, (start, end, _) in enumerate(char_spans):
            if not (pred_end <= start or pred_start >= end):  # Overlap
                predicted_indices.add(idx)

    return predicted_indices


def get_token_indices_from_ground_truth(ent_tags: list[str]) -> set[int]:
    """
    Args:
        ent_tags: List of entity tags for each token.

    Returns:
        Set of indices of tokens labeled as PII.
    """
    return {i for i, tag in enumerate(ent_tags) if tag != "O"}

In [ ]:
def evaluate_token_level(results, lst_of_tokens, lst_of_trailing_ws, lst_of_ent_tags):
    TP = FP = FN = TN = 0

    for preds, tokens, ws, labels in zip(results, lst_of_tokens, lst_of_trailing_ws, lst_of_ent_tags):
        text = reconstruct_text_from_tokens(tokens, ws, labels)  # Should be consistent with model input
        pred_set = get_token_indices_from_predictions(preds, text, tokens, ws)
        gold_set = get_token_indices_from_ground_truth(labels)

        for i in range(len(tokens)):
            if i in pred_set and i in gold_set:
                TP += 1
            elif i in pred_set and i not in gold_set:
                FP += 1
            elif i not in pred_set and i in gold_set:
                FN += 1
            else:
                TN += 1

    return {"TP": TP, "FP": FP, "FN": FN, "TN": TN}